In [1]:
pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
# Generating text with gpt2 to be used later.

def gen_text_with_gpt2(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=200, do_sample=True, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

sample_prompt = "Hello," # Feel free to change the sample_prompt
print(gen_text_with_gpt2(sample_prompt))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Hello, this is the first time I've been able to do this. It's been a long time coming, but it's finally here.

I'm really excited to get started. I'm going to be doing a lot of things right now, and I don't want to rush it. But I think it will be really interesting to see what I can do in the next couple of weeks. So I hope you guys like it as much as I do, because I really appreciate everything you do for me. And I look forward to working with you all again.


In [4]:
# A function to calculate and return the probability that text is AI-generated based on the likelihood that gpt2 is to choose a specific sequence of tokens:

def calculate_token_probabilities(text):
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    probabilities = []
    context_ids = []
    with torch.no_grad():
      for i, token_id in enumerate(token_ids):
         if i > 0:
           context_ids = token_ids[:i]
         input_ids = torch.tensor(context_ids).unsqueeze(0)
         if input_ids.nelement() == 0:
             input_ids = torch.tensor([tokenizer.bos_token_id]).unsqueeze(0)
         outputs = model(input_ids)
         logits = outputs.logits
         predicted_token_logits = logits[:, -1, :]
         probs = torch.softmax(predicted_token_logits, dim=-1)
         token_probability = probs[0, token_id].item()
         probabilities.append({
             "token": tokens[i],
             "probability": token_probability
         })
    return probabilities

In [5]:
if __name__ == '__main__':
    test_text = "Hello, this is the first time I've been able to do this. It's been a long time coming, but it's finally here." # Replace with text generated by AI earlier or directly generate text with AI using gen_text_with_gpt2("prompt/context")
    probabilities = calculate_token_probabilities(test_text)
    probabilities_total = 0
    for item in probabilities:
        print(f"Token: '{item['token']}', Probability: {item['probability']:.6f}")
        probabilities_total = probabilities_total + item['probability']
    print("Average Probability (Mean): " + str(round(probabilities_total / len(probabilities), 6))) # Returns the likelihood the text is AI-generated

Token: 'Hello', Probability: 0.000250
Token: ',', Probability: 0.096025
Token: 'Ġthis', Probability: 0.037900
Token: 'Ġis', Probability: 0.457055
Token: 'Ġthe', Probability: 0.117741
Token: 'Ġfirst', Probability: 0.128177
Token: 'Ġtime', Probability: 0.305684
Token: 'ĠI', Probability: 0.380864
Token: ''ve', Probability: 0.614528
Token: 'Ġbeen', Probability: 0.090057
Token: 'Ġable', Probability: 0.261339
Token: 'Ġto', Probability: 0.996931
Token: 'Ġdo', Probability: 0.056378
Token: 'Ġthis', Probability: 0.276099
Token: '.', Probability: 0.190753
Token: 'ĠIt', Probability: 0.065261
Token: ''s', Probability: 0.452680
Token: 'Ġbeen', Probability: 0.119649
Token: 'Ġa', Probability: 0.233669
Token: 'Ġlong', Probability: 0.256791
Token: 'Ġtime', Probability: 0.662269
Token: 'Ġcoming', Probability: 0.480321
Token: ',', Probability: 0.304317
Token: 'Ġbut', Probability: 0.514406
Token: 'Ġit', Probability: 0.135328
Token: ''s', Probability: 0.561975
Token: 'Ġfinally', Probability: 0.324781
Token: